# Parse the DrugBank XML and extract half-life of drugs to tsv files

Run using Python 3 to avoid a non-ascii character error when writing to file with the csv module.

In [1]:
import os
import csv
import gzip
import collections
import re
import io
import json
import xml.etree.ElementTree as ET

import requests
import pandas

In [2]:
xml_path = os.path.join('download', 'drugbank.xml.gz')
with gzip.open(xml_path) as xml_file:
    tree = ET.parse(xml_file)
root = tree.getroot()

In [3]:
ns = '{http://www.drugbank.ca}'
for i,drug in enumerate(root):
    row = collections.OrderedDict()
    assert drug.tag == ns + 'drug'

In [4]:
ns = '{http://www.drugbank.ca}'
# calculated properties 
cal_logp_template = "{ns}calculated-properties/{ns}property[{ns}kind='logP']/{ns}value"
cal_logs_template = "{ns}calculated-properties/{ns}property[{ns}kind='logS']/{ns}value"
cal_ws_template = "{ns}calculated-properties/{ns}property[{ns}kind='Water Solubility']/{ns}value"
cal_mw_template = "{ns}calculated-properties/{ns}property[{ns}kind='Molecular Weight']/{ns}value"
cal_psa_template = "{ns}calculated-properties/{ns}property[{ns}kind='Polar Surface Area (PSA)']/{ns}value"
cal_refr_template = "{ns}calculated-properties/{ns}property[{ns}kind='Refractivity']/{ns}value"
cal_pola_template = "{ns}calculated-properties/{ns}property[{ns}kind='Polarizability']/{ns}value"
cal_rbc_template = "{ns}calculated-properties/{ns}property[{ns}kind='Rotatable Bond Count']/{ns}value"
cal_hdac_template = "{ns}calculated-properties/{ns}property[{ns}kind='H Bond Acceptor Count']/{ns}value"
cal_hbdc_template = "{ns}calculated-properties/{ns}property[{ns}kind='H Bond Donor Count']/{ns}value"
cal_pkasa_template = "{ns}calculated-properties/{ns}property[{ns}kind='pKa (strongest acidic)']/{ns}value"
cal_pkasb_template = "{ns}calculated-properties/{ns}property[{ns}kind='pKa (strongest basic)']/{ns}value"
cal_pc_template = "{ns}calculated-properties/{ns}property[{ns}kind='Physiological Charge']/{ns}value"
cal_nr_template = "{ns}calculated-properties/{ns}property[{ns}kind='Number of Rings']/{ns}value"
cal_bioa_template = "{ns}calculated-properties/{ns}property[{ns}kind='Bioavailability']/{ns}value"
cal_rof_template = "{ns}calculated-properties/{ns}property[{ns}kind='Rule of Five']/{ns}value"
# experimental properties
exp_mw_template = "{ns}experimental-properties/{ns}property[{ns}kind='Molecular Weight']/{ns}value"
exp_logp_template = "{ns}experimental-properties/{ns}property[{ns}kind='logP']/{ns}value"
exp_pka_template = "{ns}experimental-properties/{ns}property[{ns}kind='pKa']/{ns}value"

rows = list()
for i, drug in enumerate(root):
    row = collections.OrderedDict()
    assert drug.tag == ns + 'drug'
    row['type'] = drug.get('type')
    row['drugbank_id'] = drug.findtext(ns + "drugbank-id[@primary='true']")
    row['name'] = drug.findtext(ns + "name")
    row['cal_logp'] = drug.findtext(cal_logp_template.format(ns = ns))
    row['cal_logs'] = drug.findtext(cal_logs_template.format(ns = ns))
    row['cal_water_solubility'] = drug.findtext(cal_ws_template.format(ns = ns))
    row['cal_molecular_weight'] = drug.findtext(cal_mw_template.format(ns = ns))
    row['cal_polar_surface_area'] = drug.findtext(cal_psa_template.format(ns = ns))
    row['cal_refractivity'] = drug.findtext(cal_refr_template.format(ns = ns))
    row['cal_polarizability'] = drug.findtext(cal_pola_template.format(ns = ns))
    row['cal_rotatable_bond_count'] = drug.findtext(cal_rbc_template.format(ns = ns))
    row['cal_h_bond_acceptor_count'] = drug.findtext(cal_hdac_template.format(ns = ns))
    row['cal_h_bond_donor_count'] = drug.findtext(cal_hbdc_template.format(ns = ns))
    row['cal_pka_acidic'] = drug.findtext(cal_pkasa_template.format(ns = ns))
    row['cal_pka_basic'] = drug.findtext(cal_pkasb_template.format(ns = ns))
    row['cal_physiological_charge'] = drug.findtext(cal_pc_template.format(ns = ns))
    row['cal_number_of_rings'] = drug.findtext(cal_nr_template.format(ns = ns))
    row['cal_bioavailability'] = drug.findtext(cal_bioa_template.format(ns = ns))
    row['cal_rule_of_five'] = drug.findtext(cal_rof_template.format(ns = ns))
    row['exp_molecular_weight'] = drug.findtext(exp_mw_template.format(ns = ns))
    row['exp_logp'] = drug.findtext(exp_logp_template.format(ns = ns))
    row['exp_pka'] = drug.findtext(exp_pka_template.format(ns = ns))
    row['half_life'] = drug.findtext(ns + "half-life")
    rows.append(row)

In [5]:
# combine multiple values of the same feature
def collapse_list_values(row):
    for key, value in row.items():
        if isinstance(value, list):
            row[key] = '|'.join(value)
    return row

rows = list(map(collapse_list_values, rows))
drugbank_df = pandas.DataFrame.from_dict(rows)
drugbank_df.head()

,type,drugbank_id,name,cal_logp,cal_logs,cal_water_solubility,cal_molecular_weight,cal_polar_surface_area,cal_refractivity,cal_polarizability,...,cal_pka_acidic,cal_pka_basic,cal_physiological_charge,cal_number_of_rings,cal_bioavailability,cal_rule_of_five,exp_molecular_weight,exp_logp,exp_pka,half_life
0,biotech,DB00001,Lepirudin,None,None,None,None,None,None,None,...,None,None,None,None,None,None,6963.4250,None,None,Approximately 1.3 hours
1,biotech,DB00002,Cetuximab,None,None,None,None,None,None,None,...,None,None,None,None,None,None,145781.6000,None,None,114 hrs
2,biotech,DB00003,Dornase alfa,None,None,None,None,None,None,None,...,None,None,None,None,None,None,29253.9000,None,None,
3,biotech,DB00004,Denileukin diftitox,None,None,None,None,None,None,None,...,None,None,None,None,None,None,57647.3000,None,None,70-80 min
4,biotech,DB00005,Etanercept,None,None,None,None,None,None,None,...,None,None,None,None,None,None,51234.9000,None,None,102 +/- 30 hrs in individuals with rheumatoid ...


In [6]:
# output extracted features to tsv file
drugbank_df.to_csv(path_or_buf='data/drugbank_halflife.tsv', sep='\t', index=False)

# Summary statistics of extracted structural information

In [7]:
# The half life information is in free text & various format
drugbank_df['half_life'].value_counts()

                                                                                                                                                                                           6439
2 hours                                                                                                                                                                                      16
1 hour                                                                                                                                                                                       12
10 hours                                                                                                                                                                                     12
3 hours                                                                                                                                                                                       8
6-8 hours                               

In [8]:
# number of extracted drugs 
len(drugbank_df)

7759

In [9]:
# number of extracted drugs with half life information
len(drugbank_df) - sum(drugbank_df['half_life'] == '')

1320

In [10]:
# number of extracted drugs with other structural information
len(drugbank_df) - drugbank_df.isin([None]).sum(axis=0)

type                         7759
drugbank_id                  7759
name                         7759
cal_logp                     6866
cal_logs                     6746
cal_water_solubility         6746
cal_molecular_weight         6870
cal_polar_surface_area       6868
cal_refractivity             6868
cal_polarizability           6863
cal_rotatable_bond_count     6868
cal_h_bond_acceptor_count    6868
cal_h_bond_donor_count       6868
cal_pka_acidic               5890
cal_pka_basic                6438
cal_physiological_charge     6868
cal_number_of_rings          6868
cal_bioavailability          6868
cal_rule_of_five             5433
exp_molecular_weight          153
exp_logp                     1391
exp_pka                       417
half_life                    7759
dtype: int64

In [11]:
# drugs with logP, MW, and solubility information. These features can be used to predict half-life of a drug
other_info = drugbank_df.isin([None])['cal_logp'] & drugbank_df.isin([None])['cal_water_solubility'] & drugbank_df.isin([None])['cal_molecular_weight']
drugbank_df1 = drugbank_df[~other_info]
drugbank_df1.head()

,type,drugbank_id,name,cal_logp,cal_logs,cal_water_solubility,cal_molecular_weight,cal_polar_surface_area,cal_refractivity,cal_polarizability,...,cal_pka_acidic,cal_pka_basic,cal_physiological_charge,cal_number_of_rings,cal_bioavailability,cal_rule_of_five,exp_molecular_weight,exp_logp,exp_pka,half_life
13,small molecule,DB00014,Goserelin,0.3,-4.7,2.83e-02 g/l,1269.4105,495.89,325.84,131.22,...,9.27,10.82,2,6,0,None,None,-2,None,4-5 hours
34,small molecule,DB00035,Desmopressin,-1,-4,1.10e-01 g/l,1069.217,435.41,279.78,106.19,...,9.5,11.77,1,4,0,None,None,-4.2,None,Oral t<sub>1/2</sub>=1.5-2.5 hours. Intranasal...
48,small molecule,DB00050,Cetrorelix,1.33,-5.3,6.94e-03 g/l,1431.038,495.67,384.16,148.09,...,9.49,11.11,1,6,0,None,None,None,None,~62.8 hours
86,small molecule,DB00091,Cyclosporine,4.12,-5.1,9.52e-03 g/l,1202.6112,278.8,327.14,133.24,...,11.83,-2.4,0,1,0,None,None,None,None,"Biphasic and variable, approximately 7 hours (..."
88,small molecule,DB00093,Felypressin,-1.1,-4.4,4.53e-02 g/l,1040.219,405.32,264.79,103.93,...,11.39,10.18,2,4,0,None,None,None,None,


In [12]:
# number of extracted drugs with logP, MW, solubility, and half life information
len(drugbank_df1) - sum(drugbank_df1['half_life'] == '')

1145